# XAI Recipes for the HuggingFace 🤗 Image Classification Models
# ![DFF](https://raw.githubusercontent.com/jacobgil/pytorch-grad-cam/master/tutorials/huggingface_dff.png)

In [1]:
# !pip install pytorch-gradcam
# !pip install git+https://github.com/jacobgil/pytorch-grad-cam.git
# !git lfs clone https://github.com/jacobgil/pytorch-grad-cam.git

In [2]:
import warnings
warnings.filterwarnings('ignore')

import sys

# for run_dff_on_image
sys.path.insert(1, '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/play/pytorch-grad-cam/pytorch_grad_cam/feature_factorization')
import deep_feature_factorization

# for ClassifierOutputTarget
sys.path.insert(1, '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/play/pytorch-grad-cam/pytorch_grad_cam/utils/')
import model_targets

sys.path.insert(1, '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/play/pytorch-grad-cam/pytorch_grad_cam/')
import base_cam

import image
import cv2
import torch

import numpy as np

from PIL import Image
from gradcam import GradCAM
from datasets import load_dataset
from torchvision import transforms
from typing import List, Callable, Optional

ModuleNotFoundError: No module named 'deep_feature_factorization'

In [ ]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]
img_tensor = transforms.ToTensor()(image)

In [ ]:
""" Model wrapper to return a tensor"""
class HuggingfaceToTensorModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingfaceToTensorModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

In [ ]:
""" Translate the category name to the category index.
    Some models aren't trained on Imagenet but on even larger datasets,
    so we can't just assume that 761 will always be remote-control.

"""
def category_name_to_index(model, category_name):
    name_to_index = dict((v, k) for k, v in model.config.id2label.items())
    return name_to_index[category_name]

In [ ]:
""" Helper function to run GradCAM on an image and create a visualization.
    (note to myself: this is probably useful enough to move into the package)
    If several targets are passed in targets_for_gradcam,
    e.g different categories,
    a visualization for each of them will be created.
    
# """
# def run_grad_cam_on_image(model: torch.nn.Module,
#                           target_layer: torch.nn.Module,
#                           targets_for_gradcam: List[Callable],
#                           reshape_transform: Optional[Callable],
#                           input_tensor: torch.nn.Module=img_tensor,
#                           input_image: Image=image,
#                           method: Callable=GradCAM):
#     with method(model=HuggingfaceToTensorModelWrapper(model),
#                  target_layers=[target_layer],
#                  reshape_transform=reshape_transform) as cam:

#         # Replicate the tensor for each of the categories we want to create Grad-CAM for:
#         repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

#         batch_results = cam(input_tensor=repeated_tensor,
#                             targets=targets_for_gradcam)
#         results = []
#         for grayscale_cam in batch_results:
#             visualization = image.show_cam_on_image(np.float32(input_image)/255,
#                                               grayscale_cam,
#                                               use_rgb=True)
#             # Make it weight less in the notebook:
#             visualization = cv2.resize(visualization,
#                                        (visualization.shape[1]//2, visualization.shape[0]//2))
#             results.append(visualization)
#         return np.hstack(results)

In [ ]:
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

In [ ]:
from transformers import ResNetForImageClassification
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# We will show GradCAM for the "Egyptian Cat" and the 'Remote Control" categories:
targets_for_gradcam = [model_targets.ClassifierOutputTarget(category_name_to_index(model, "Egyptian cat")),
                       model_targets.ClassifierOutputTarget(category_name_to_index(model, "remote control, remote"))]

# The last layer in the Resnet Encoder:
target_layer = model.resnet.encoder.stages[-1].layers[-1]

display(Image.fromarray(deep_feature_factorization.run_dff_on_image(model=model,
                          target_layer=target_layer,
                          classifier=model.classifier,
                          img_pil=image,
                          img_tensor=img_tensor,
                          reshape_transform=None,
                          n_components=4,
                          top_k=2)))
# display(Image.fromarray(run_grad_cam_on_image(model=model,
#                       target_layer=target_layer,
#                       targets_for_gradcam=targets_for_gradcam,
#                       reshape_transform=None)))
print_top_categories(model, img_tensor)

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
def reshape_transform_vit_huggingface(x):
    activations = x[:, 1:, :]
    activations = activations.view(activations.shape[0],
                                   12, 12, activations.shape[2])
    activations = activations.transpose(2, 3).transpose(1, 2)
    return activations

model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384')
targets_for_gradcam = [model_targets.ClassifierOutputTarget(category_name_to_index(model, "Egyptian cat")),
                       model_targets.ClassifierOutputTarget(category_name_to_index(model, "remote control, remote"))]
target_layer_dff = model.vit.layernorm
target_layer_gradcam = model.vit.encoder.layer[-2].output
image_resized = image.resize((384, 384))
tensor_resized = transforms.ToTensor()(image_resized)


display(Image.fromarray(deep_feature_factorization.run_dff_on_image(model=model,
                          target_layer=target_layer_dff,
                          classifier=model.classifier,
                          img_pil=image_resized,
                          img_tensor=tensor_resized,
                          reshape_transform=reshape_transform_vit_huggingface,
                          n_components=4,
                          top_k=2)))
display(Image.fromarray(run_grad_cam_on_image(model=model,
                      target_layer=target_layer_gradcam,
                      targets_for_gradcam=targets_for_gradcam,
                      input_tensor=tensor_resized,
                      input_image=image_resized,
                      reshape_transform=reshape_transform_vit_huggingface)))
print_top_categories(model, tensor_resized)